In [15]:
%reload_ext autoreload
%autoreload 2

from IPython.core.display import Markdown
from tqdm.auto import tqdm
import config
import docs
import pandas as pd
import requests

pd.set_option('display.max_colwidth', None)

INDICATOR = 'water_sources'
CONFIG = config.get_config(INDICATOR, '../config.toml')

display(Markdown('## Raw data path'))
raw_dir_path = str(CONFIG['raw_dir']).replace('\\', '/')
display(Markdown(f"[{raw_dir_path}]({raw_dir_path})"))


## Raw data path

[C:/Users/tan/src/regional-pm-2023/data/raw/utilities/water/water_sources](C:/Users/tan/src/regional-pm-2023/data/raw/utilities/water/water_sources)

# Environment: Water

## Water Sources

In [16]:
docs.describe_indicator(
    indicator=INDICATOR,
    indicators_xlsx_path=CONFIG['indicators_xlsx_path'],
)

Percentages of different water sources.

nan

In [17]:
docs.list_schema(
    indicator=INDICATOR,
    indicators_xlsx_path=CONFIG['indicators_xlsx_path'],
)

,name,description,type
column,,,
year,Fiscal Year,Fiscal year of record.,datetime
source,Source,Water source of record.,string
percentage,Percentage,"Percentage of water source used, in a given year for a given source.",float


In [18]:
docs.list_sources(
    indicator=INDICATOR,
    indicators_xlsx_path=CONFIG['indicators_xlsx_path'],
)

,name,organization,active,notes
source,,,,
cwa_budget,Annual Budget Reports,San Diego County Water Authority,True,Based on fiscal year instead of calendar year.


In [19]:
steps = docs.list_update_steps(
    indicator=INDICATOR,
    indicators_xlsx_path=CONFIG['indicators_xlsx_path'],
)
steps

,step
0,Update legacy sheet based on newest CWA fiscal year report.
1,Extract legacy data from legacy PM sheet.


In [20]:
docs.list_remarks(
    indicator=INDICATOR,
    indicators_xlsx_path=CONFIG['indicators_xlsx_path'],
)

,author,note
0,TAN,We have not identified a current SANDAG SME for this.


## Step 0: Update legacy sheet

2022 Central Water Authority's [Fiscal Year report](https://www.sdcwa.org/wp-content/uploads/2020/11/FY22-ACFR-Electronic11.08.22SECURED.pdf), from the chart on page 9.

In [21]:
display(steps.loc[0])

step    Update legacy sheet based on newest CWA fiscal year report.
Name: 0, dtype: object

## Step 1: Extract from legacy sheet

In [22]:
water_sources = (
    pd.read_excel(
        CONFIG['legacy_xlsx_path'],
        CONFIG['legacy_sheet'],
        usecols='A:R',
        skiprows=3,
        nrows=11,
    )
    .rename(columns={'Unnamed: 0': 'source'})
    .melt(id_vars='source', var_name='year', value_name='percentage')
    .assign(source=lambda df: df.source.str.strip('*'))
    .assign(year=lambda df: pd.to_datetime(df.year, format='%Y'))
    .set_index(['year', 'source'])
)
display(water_sources.tail(3))

percentage
year       source                         
2021-01-01 Local Surface Water        0.09
           Dry-Year Transfer          0.00
           Potable Reuse              0.00

### Save Data

In [23]:
display(Markdown('#### Clean data path'))
clean_dir_path = str(CONFIG['clean_dir']).replace('\\', '/')
display(Markdown(f"[{clean_dir_path}]({clean_dir_path})"))

#### Clean data path

[C:/Users/tan/src/regional-pm-2023/data/clean/utilities/water/water_sources](C:/Users/tan/src/regional-pm-2023/data/clean/utilities/water/water_sources)

In [25]:
water_sources.to_csv(
    CONFIG['clean_dir']
    / f'{INDICATOR}_odp.csv'
)